In [1]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [142]:
out='../output/'
inp='../input/monthly/'
pref='idm'

In [143]:
dat=pd.DataFrame(columns=[0,1,2])
onlyfiles = [f for f in listdir(inp) if isfile(join(inp, f))]
for f in onlyfiles:
    if (f.split('_')[0]==pref):
        #print(f)
        with open(join(inp, f), "r") as read_file:
            data = pd.DataFrame(json.load(read_file))
            dat=pd.concat([dat,data])

dat

,0,1,2
0,Date:Time,Price(USD/MWh),Turnover(MWh)
1,(2020/01/01) 00-01,30,75
2,(2020/01/01) 01-02,30,70
3,(2020/01/01) 02-03,0,0
4,(2020/01/01) 03-04,30,65
...,...,...,...
600,(2022/08/25) 23-24,0,0
601,Min,0,0
602,Max,300,308
603,Average,71.03235,70.956666666666666666666666667


In [144]:
dat.drop(dat[dat[0] == 'Date:Time'].index, inplace = True)
dat.drop(dat[dat[0] == 'Min'].index, inplace = True)
dat.drop(dat[dat[0] == 'Max'].index, inplace = True)
dat.drop(dat[dat[0] == 'Average'].index, inplace = True)
dat.drop(dat[dat[0] == 'Total'].index, inplace = True)
dat.columns=['datehour','price','volume']
dat['price']=dat['price'].astype(float)
dat['volume']=dat['volume'].astype(float)
dat

,datehour,price,volume
1,(2020/01/01) 00-01,30.0,75.0
2,(2020/01/01) 01-02,30.0,70.0
3,(2020/01/01) 02-03,0.0,0.0
4,(2020/01/01) 03-04,30.0,65.0
5,(2020/01/01) 04-05,0.0,0.0
...,...,...,...
596,(2022/08/25) 19-20,0.0,0.0
597,(2022/08/25) 20-21,95.0,40.0
598,(2022/08/25) 21-22,0.0,0.0
599,(2022/08/25) 22-23,40.0,145.0


In [145]:
dat['date']=pd.to_datetime(dat.datehour.str.slice(1,11),format="%Y/%m/%d")
dat['hour']=dat.datehour.str.slice(13,15).astype(int)
dat

,datehour,price,volume,date,hour
1,(2020/01/01) 00-01,30.0,75.0,2020-01-01,0
2,(2020/01/01) 01-02,30.0,70.0,2020-01-01,1
3,(2020/01/01) 02-03,0.0,0.0,2020-01-01,2
4,(2020/01/01) 03-04,30.0,65.0,2020-01-01,3
5,(2020/01/01) 04-05,0.0,0.0,2020-01-01,4
...,...,...,...,...,...
596,(2022/08/25) 19-20,0.0,0.0,2022-08-25,19
597,(2022/08/25) 20-21,95.0,40.0,2022-08-25,20
598,(2022/08/25) 21-22,0.0,0.0,2022-08-25,21
599,(2022/08/25) 22-23,40.0,145.0,2022-08-25,22


In [146]:
dat=dat.drop(['datehour'],1)
dat

,price,volume,date,hour
1,30.0,75.0,2020-01-01,0
2,30.0,70.0,2020-01-01,1
3,0.0,0.0,2020-01-01,2
4,30.0,65.0,2020-01-01,3
5,0.0,0.0,2020-01-01,4
...,...,...,...,...
596,0.0,0.0,2022-08-25,19
597,95.0,40.0,2022-08-25,20
598,0.0,0.0,2022-08-25,21
599,40.0,145.0,2022-08-25,22


In [147]:
hourly=dat.set_index(pd.DatetimeIndex(dat['date']))
hourly['year']=hourly.index.year
hourly['month']=hourly.index.month
hourly['day']=hourly.index.day
hourly['day_of_week']=hourly.index.dayofweek
hourly=hourly.drop(['date'],axis=1)
hourly

,price,volume,hour,year,month,day,day_of_week
date,,,,,,,
2020-01-01,30.0,75.0,0,2020,1,1,2
2020-01-01,30.0,70.0,1,2020,1,1,2
2020-01-01,0.0,0.0,2,2020,1,1,2
2020-01-01,30.0,65.0,3,2020,1,1,2
2020-01-01,0.0,0.0,4,2020,1,1,2
...,...,...,...,...,...,...,...
2022-08-25,0.0,0.0,19,2022,8,25,3
2022-08-25,95.0,40.0,20,2022,8,25,3
2022-08-25,0.0,0.0,21,2022,8,25,3


In [148]:
hourly['value']=hourly['price']*hourly['volume']
daily=hourly.reset_index().groupby(['year','month','day']).sum().drop(['hour','day_of_week'],axis=1)
daily['price_weight_av']=daily['value']/daily['volume']
daily=daily.drop(['price'], axis=1)
daily['price_max']=hourly[['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_min']=hourly[['year','month','day','price']].groupby(['year','month','day']).min()
daily['price_mean']=hourly[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['power_max']=hourly[['year','month','day','volume']].groupby(['year','month','day']).max()
daily['power_min']=hourly[['year','month','day','volume']].groupby(['year','month','day']).min()
daily['power_mean']=hourly[['year','month','day','volume']].groupby(['year','month','day']).mean()

daily=daily.reset_index()
daily['date']=pd.to_datetime(daily.reset_index()[['year', 'month', 'day']])
daily=daily.set_index(['year','month','day'])
daily


volume      value  price_weight_av  price_max  price_min  \
year month day                                                             
2020 1     1     826.0   38450.32        46.550024      100.0        0.0   
           2    1336.0   69778.31        52.229274      105.0        0.0   
           3    1245.0   71393.85        57.344458       85.0        0.0   
           4    1149.0   37270.00        32.436902       65.0        0.0   
           5     756.0   22680.00        30.000000       30.0        0.0   
...                ...        ...              ...        ...        ...   
2022 8     21      0.0       0.00              NaN        0.0        0.0   
           22   1968.0  228570.79       116.143694      250.0        0.0   
           23    627.0   44772.90        71.408134      250.0        0.0   
           24    933.0   74207.40        79.536334      300.0        0.0   
           25    245.0   22800.00        93.061224      220.0        0.0   

                price_mean  power_max  power_min  power_mean       date  
year month day                                                           
2020 1     1     30.092917       85.0        0.0   34.416667 2020-01-01  
           2     33.130417      173.0        0.0   55.666667 2020-01-02  
           3     28.745417      246.0        0.0   51.875000 2020-01-03  
           4     24.166667      130.0        0.0   47.875000 2020-01-04  
           5     18.750000      142.0        0.0   31.500000 2020-01-05  
...                    ...        ...        ...         ...        ...  
2022 8     21     0.000000        0.0        0.0    0.000000 2022-08-21  
           22    95.802083      181.0        0.0   82.000000 2022-08-22  
           23    52.679167      139.0        0.0   26.125000 2022-08-23  
           24    80.229167      145.0        0.0   38.875000 2022-08-24  
           25    14.791667      145.0        0.0   10.208333 2022-08-25  

[942 rows x 10 columns]

In [150]:
hourly['cat']='off'

if (pref=='dam' or pref=='idm'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==6),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=7) & (hourly['hour']<=9),'cat']='morning'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=10) & (hourly['hour']<=16),'cat']='standard'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & ((hourly['month']<=5) | (hourly['month']>=9)),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']<=3),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==19),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==21),'cat']='standard'

    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==19),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='standard'

if (pref=='fpm'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=21),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'

if (pref=='fpw'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==5),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=8),'cat']='morning'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=9) & (hourly['hour']<=17),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='evening'
    
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'
    
hourly

,price,volume,hour,year,month,day,day_of_week,value,cat
date,,,,,,,,,
2020-01-01,30.0,75.0,0,2020,1,1,2,2250.0,off
2020-01-01,30.0,70.0,1,2020,1,1,2,2100.0,off
2020-01-01,0.0,0.0,2,2020,1,1,2,0.0,off
2020-01-01,30.0,65.0,3,2020,1,1,2,1950.0,off
2020-01-01,0.0,0.0,4,2020,1,1,2,0.0,off
...,...,...,...,...,...,...,...,...,...
2022-08-25,0.0,0.0,19,2022,8,25,3,0.0,evening
2022-08-25,95.0,40.0,20,2022,8,25,3,3800.0,standard
2022-08-25,0.0,0.0,21,2022,8,25,3,0.0,standard


In [151]:
daily['off_price_mean']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['standard_price_mean']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['morning_price_mean']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()
daily['evening_price_mean']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['off_hour_count']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['standard_hour_count']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['morning_hour_count']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()
daily['evening_hour_count']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','price']].groupby(['year','month','day']).count()

daily['off_volume']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['standard_volume']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['morning_volume']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['evening_volume']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','volume']].groupby(['year','month','day']).sum()

daily['off_value']=hourly.loc[hourly['cat']=='off'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['standard_value']=hourly.loc[hourly['cat']=='standard'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['morning_value']=hourly.loc[hourly['cat']=='morning'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()
daily['evening_value']=hourly.loc[hourly['cat']=='evening'].reset_index()[['year','month','day','value']].groupby(['year','month','day']).sum()

daily

volume      value  price_weight_av  price_max  price_min  \
year month day                                                             
2020 1     1     826.0   38450.32        46.550024      100.0        0.0   
           2    1336.0   69778.31        52.229274      105.0        0.0   
           3    1245.0   71393.85        57.344458       85.0        0.0   
           4    1149.0   37270.00        32.436902       65.0        0.0   
           5     756.0   22680.00        30.000000       30.0        0.0   
...                ...        ...              ...        ...        ...   
2022 8     21      0.0       0.00              NaN        0.0        0.0   
           22   1968.0  228570.79       116.143694      250.0        0.0   
           23    627.0   44772.90        71.408134      250.0        0.0   
           24    933.0   74207.40        79.536334      300.0        0.0   
           25    245.0   22800.00        93.061224      220.0        0.0   

                price_mean  power_max  power_min  power_mean       date  ...  \
year month day                                                           ...   
2020 1     1     30.092917       85.0        0.0   34.416667 2020-01-01  ...   
           2     33.130417      173.0        0.0   55.666667 2020-01-02  ...   
           3     28.745417      246.0        0.0   51.875000 2020-01-03  ...   
           4     24.166667      130.0        0.0   47.875000 2020-01-04  ...   
           5     18.750000      142.0        0.0   31.500000 2020-01-05  ...   
...                    ...        ...        ...         ...        ...  ...   
2022 8     21     0.000000        0.0        0.0    0.000000 2022-08-21  ...   
           22    95.802083      181.0        0.0   82.000000 2022-08-22  ...   
           23    52.679167      139.0        0.0   26.125000 2022-08-23  ...   
           24    80.229167      145.0        0.0   38.875000 2022-08-24  ...   
           25    14.791667      145.0        0.0   10.208333 2022-08-25  ...   

                morning_hour_count  evening_hour_count  off_volume  \
year month day                                                       
2020 1     1                   3.0                 2.0       339.0   
           2                   3.0                 2.0       290.0   
           3                   3.0                 2.0        33.0   
           4                   NaN                 NaN      1129.0   
           5                   NaN                 NaN       756.0   
...                            ...                 ...         ...   
2022 8     21                  NaN                 NaN         0.0   
           22                  3.0                 3.0       239.0   
           23                  3.0                 3.0       280.0   
           24                  3.0                 3.0       470.0   
           25                  3.0                 3.0       145.0   

                standard_volume  morning_volume  evening_volume  off_value  \
year month day                                                               
2020 1     1              457.0            30.0             0.0    10170.0   
           2             1046.0             0.0             0.0     8700.0   
           3             1062.0           150.0             0.0      990.0   
           4               20.0             NaN             NaN    35970.0   
           5                NaN             NaN             NaN    22680.0   
...                         ...             ...             ...        ...   
2022 8     21               NaN             NaN             NaN        0.0   
           22            1241.0           169.0           319.0     9610.0   
           23             288.0            30.0            29.0    13894.9   
           24             269.0           101.0            93.0    17325.4   
           25              40.0            60.0             0.0     5800.0   

                standard_value  morning_value  evening_value

In [152]:
daily.to_csv(out + pref + '_daily.csv')

In [153]:
hourly.to_csv(out + pref + '_hourly.csv')

In [154]:
hourly_flat=hourly.reset_index().set_index(['date'])
hourly_slim=hourly_flat[['hour','price','volume','cat']]
hourly_slim

,hour,price,volume,cat
date,,,,
2020-01-01,0,30.0,75.0,off
2020-01-01,1,30.0,70.0,off
2020-01-01,2,0.0,0.0,off
2020-01-01,3,30.0,65.0,off
2020-01-01,4,0.0,0.0,off
...,...,...,...,...
2022-08-25,19,0.0,0.0,evening
2022-08-25,20,95.0,40.0,standard
2022-08-25,21,0.0,0.0,standard


In [155]:
hourly_slim.to_csv(out + pref + '_hourly_slim.csv')

In [156]:
monthly=daily.groupby(['year','month']).mean().reset_index(['month','year'])
monthly=monthly.drop(['morning_volume','off_volume','standard_volume','evening_volume','price_weight_av'],1)
monthly.rename(columns={'value':'value_mean','volume':'volume_mean','off_hour_count':'off_hour_mean','standard_hour_count':'standard_hour_mean','morning_hour_count':'morning_hour_mean','evening_hour_count':'evening_hour_mean'}, inplace=True)
monthly[['price_max','standard_price_max','off_price_max','morning_price_max','evening_price_max']]=daily.groupby(['year','month']).max().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_min','standard_price_min','off_price_min','morning_price_min','evening_price_min']]=daily.groupby(['year','month']).min().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_stdev','standard_price_stdev','off_price_stdev','morning_price_stdev','evening_price_stdev']]=daily.groupby(['year','month']).std().reset_index(['month','year'])[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]

monthly[['volume','morning_volume','evening_volume','standard_volume','off_volume']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['volume','morning_volume','evening_volume','standard_volume','off_volume']]
monthly[['value','morning_value','evening_value','standard_value','off_value']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['value','morning_value','evening_value','standard_value','off_value']]

monthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=daily.groupby(['year','month']).sum().reset_index(['month','year'])[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]

monthly['price_weight_mean']=monthly['value'] / monthly['volume']
monthly['off_price_weight_mean']=monthly['off_value'] / monthly['off_volume']
monthly['standard_price_weight_mean']=monthly['standard_value'] / monthly['standard_volume']
monthly['morning_price_weight_mean']=monthly['morning_value'] / monthly['morning_volume']
monthly['evening_price_weight_mean']=monthly['evening_value'] / monthly['evening_volume']


monthly['off_power']=monthly['off_volume'] / monthly['off_hour_count']
monthly['morning_power']=monthly['morning_volume'] / monthly['morning_hour_count']
monthly['standard_power']=monthly['standard_volume'] / monthly['standard_hour_count']
monthly['evening_power']=monthly['evening_volume'] / monthly['evening_hour_count']

monthly

,year,month,volume_mean,value_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2020,1,991.354839,53644.180968,62.000000,0.0,30.185462,129.129032,0.000000,42.681452,...,46.0,54.111988,32.108663,64.906243,80.040513,85.233918,35.882716,34.159420,57.170819,14.869565
1,2020,2,297.620690,15747.605862,46.041667,0.0,14.539856,47.931034,0.689655,12.816667,...,40.0,52.911664,30.720552,69.224777,85.000000,85.623188,12.212308,0.750000,15.400000,17.250000
2,2020,3,507.645161,29668.449677,61.077500,0.0,23.620484,61.000000,1.290323,22.660215,...,44.0,58.443283,29.703441,63.369498,82.994055,97.889149,13.250000,35.015152,29.951852,18.954545
3,2020,4,658.900000,23955.040000,43.063750,0.0,17.609903,76.333333,3.500000,27.913056,...,44.0,36.356109,23.215252,35.603486,53.411492,72.083612,20.271875,29.136364,35.870370,38.000000
4,2020,5,466.645161,15793.785161,35.041667,0.0,12.016694,54.516129,4.032258,19.865591,...,42.0,33.845385,24.345620,39.389279,50.286008,82.774892,18.171920,7.555556,27.883459,5.500000
5,2020,6,242.033333,12947.578333,58.800000,0.0,13.673514,48.700000,0.000000,10.440000,...,66.0,53.495021,20.319950,50.900064,59.951392,111.301964,2.522152,10.560606,20.460317,9.257576
6,2020,7,598.354839,29771.077742,49.967000,0.0,27.719019,91.064516,0.000000,26.231720,...,69.0,49.754888,19.661564,49.353281,65.314062,124.972321,11.228125,64.289855,37.965649,8.304348
7,2020,8,737.193548,36092.595806,53.452500,0.0,24.246487,85.129032,1.387097,32.101075,...,63.0,48.959457,20.854453,47.707332,61.719536,117.634118,13.002907,54.015873,54.128000,22.936508
8,2020,9,1096.666667,47867.866667,51.950000,0.0,27.804139,123.133333,0.533333,47.210278,...,44.0,43.648511,22.930684,47.449751,59.971471,90.000000,26.878125,73.939394,69.770370,13.204545
9,2020,10,1266.483871,59794.354516,66.250000,0.0,31.968288,135.580645,3.258065,53.543548,...,44.0,47.212883,25.164186,56.729911,65.490669,97.754583,45.774775,65.954545,64.888087,38.431818


In [157]:
if (pref=='dam'):
    monthly['evening_days']=daily.loc[daily['evening_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['morning_days']=daily.loc[daily['morning_hour_count']>0].groupby(['year','month']).count().reset_index(['month','year'])['volume']
    monthly['days']=daily.groupby(['year','month']).count().reset_index(['month','year'])['volume']

monthly

,year,month,volume_mean,value_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,...,evening_hour_count,price_weight_mean,off_price_weight_mean,standard_price_weight_mean,morning_price_weight_mean,evening_price_weight_mean,off_power,morning_power,standard_power,evening_power
0,2020,1,991.354839,53644.180968,62.000000,0.0,30.185462,129.129032,0.000000,42.681452,...,46.0,54.111988,32.108663,64.906243,80.040513,85.233918,35.882716,34.159420,57.170819,14.869565
1,2020,2,297.620690,15747.605862,46.041667,0.0,14.539856,47.931034,0.689655,12.816667,...,40.0,52.911664,30.720552,69.224777,85.000000,85.623188,12.212308,0.750000,15.400000,17.250000
2,2020,3,507.645161,29668.449677,61.077500,0.0,23.620484,61.000000,1.290323,22.660215,...,44.0,58.443283,29.703441,63.369498,82.994055,97.889149,13.250000,35.015152,29.951852,18.954545
3,2020,4,658.900000,23955.040000,43.063750,0.0,17.609903,76.333333,3.500000,27.913056,...,44.0,36.356109,23.215252,35.603486,53.411492,72.083612,20.271875,29.136364,35.870370,38.000000
4,2020,5,466.645161,15793.785161,35.041667,0.0,12.016694,54.516129,4.032258,19.865591,...,42.0,33.845385,24.345620,39.389279,50.286008,82.774892,18.171920,7.555556,27.883459,5.500000
5,2020,6,242.033333,12947.578333,58.800000,0.0,13.673514,48.700000,0.000000,10.440000,...,66.0,53.495021,20.319950,50.900064,59.951392,111.301964,2.522152,10.560606,20.460317,9.257576
6,2020,7,598.354839,29771.077742,49.967000,0.0,27.719019,91.064516,0.000000,26.231720,...,69.0,49.754888,19.661564,49.353281,65.314062,124.972321,11.228125,64.289855,37.965649,8.304348
7,2020,8,737.193548,36092.595806,53.452500,0.0,24.246487,85.129032,1.387097,32.101075,...,63.0,48.959457,20.854453,47.707332,61.719536,117.634118,13.002907,54.015873,54.128000,22.936508
8,2020,9,1096.666667,47867.866667,51.950000,0.0,27.804139,123.133333,0.533333,47.210278,...,44.0,43.648511,22.930684,47.449751,59.971471,90.000000,26.878125,73.939394,69.770370,13.204545
9,2020,10,1266.483871,59794.354516,66.250000,0.0,31.968288,135.580645,3.258065,53.543548,...,44.0,47.212883,25.164186,56.729911,65.490669,97.754583,45.774775,65.954545,64.888087,38.431818


In [158]:
monthly=monthly.set_index(['year','month'])

In [159]:
monthly['off_price_diff']=monthly['off_price_mean']-monthly['off_price_mean'].shift(1)
monthly['morning_price_diff']=monthly['morning_price_mean']-monthly['morning_price_mean'].shift(1)
monthly['standard_price_diff']=monthly['standard_price_mean']-monthly['standard_price_mean'].shift(1)
monthly['evening_price_diff']=monthly['evening_price_mean']-monthly['evening_price_mean'].shift(1)

monthly['off_price_log_diff']=np.log10(monthly['off_price_mean'])-np.log10(monthly['off_price_mean']).shift(1)
monthly['morning_price_log_diff']=np.log10(monthly['morning_price_mean'])-np.log10(monthly['morning_price_mean']).shift(1)
monthly['standard_price_log_diff']=np.log10(monthly['standard_price_mean'])-np.log10(monthly['standard_price_mean']).shift(1)
monthly['evening_price_log_diff']=np.log10(monthly['evening_price_mean'])-np.log10(monthly['evening_price_mean']).shift(1)

In [160]:
monthly.to_csv(out + pref + '_monthly.csv')

In [161]:
monthly_filt=monthly #.loc[monthly['year']<2020]
daily_filt=daily.reset_index() #.loc[daily.reset_index()['year']<2020]

In [162]:
calmonthly=daily_filt.groupby(['month']).mean().reset_index()
calmonthly=calmonthly.drop(['year','day','volume','standard_volume','off_volume','morning_volume','evening_volume', 'off_hour_count', 'morning_hour_count', 'standard_hour_count','evening_hour_count'],1)
calmonthly.rename(columns={'value':'value_mean','volume':'volume_mean','price_weight_av': 'price_weight_mean'}, inplace=True)

calmonthly[['price_median', 'off_price_median','standard_price_median', 'morning_price_median', 'evening_price_median']]=daily_filt.groupby(['month']).median().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]
calmonthly[['price_stdev', 'off_price_stdev','standard_price_stdev', 'morning_price_stdev', 'evening_price_stdev']]=daily_filt.groupby(['month']).std().reset_index()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]

calmonthly[['volume', 'off_volume','standard_volume', 'morning_volume', 'evening_volume']]=monthly_filt.groupby(['month']).mean().reset_index()[['volume','off_volume','standard_volume', 'morning_volume', 'evening_volume']]
calmonthly[['value', 'off_value','standard_value', 'morning_value', 'evening_value']]=monthly_filt.groupby(['month']).mean().reset_index()[['value','off_value','standard_value', 'morning_value', 'evening_value']]
calmonthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]
calmonthly[['off_power','standard_power', 'morning_power', 'evening_power']]=monthly_filt.groupby(['month']).mean().reset_index()[['off_power','standard_power', 'morning_power', 'evening_power']]

calmonthly

,month,value_mean,price_weight_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,off_price_mean,...,evening_volume,value,off_hour_count,standard_hour_count,morning_hour_count,evening_hour_count,off_power,standard_power,morning_power,evening_power
0,1,42288.783548,53.219988,69.512688,0.376344,27.496830,102.075269,0.107527,33.227957,12.600314,...,705.333333,1.310952e+06,340.666667,271.000000,65.0,43.333333,25.462338,41.954183,48.386473,16.321601
1,2,20641.688235,54.285043,56.226471,0.282353,16.479260,61.341176,0.235294,15.619216,9.260001,...,514.666667,5.848478e+05,316.333333,250.333333,60.0,40.000000,11.272692,21.097043,17.966667,12.866667
2,3,23468.080968,51.143816,57.015269,0.763441,19.230035,62.225806,1.301075,19.447670,11.262188,...,796.666667,7.275105e+05,329.333333,277.333333,68.0,45.333333,12.462963,22.320843,35.420509,17.593544
3,4,21492.963596,56.394161,65.734382,0.872697,18.644998,58.932584,1.516854,18.536517,11.022931,...,1073.666667,6.376246e+05,321.666667,265.000000,64.0,42.666667,11.435160,25.078359,22.698990,25.332576
4,5,14842.413103,54.718191,65.662069,0.528736,13.226426,49.471264,1.436782,13.294397,5.518835,...,445.333333,4.304300e+05,328.333333,250.333333,60.0,40.000000,8.545061,19.311795,14.716138,11.808201
5,6,34716.919770,84.430141,124.356322,1.737816,31.559901,61.390805,0.666667,15.231992,14.801944,...,1365.666667,1.006791e+06,312.000000,242.000000,63.0,63.000000,8.042583,21.798263,16.635832,22.039607
6,7,33862.700909,79.334481,123.097792,0.000000,31.130118,75.177922,0.000000,20.993398,12.741431,...,948.200000,8.691427e+05,272.000000,216.000000,56.0,56.000000,10.156181,23.999152,31.722881,17.854985
7,8,88106.594828,73.581028,130.152414,1.471264,41.604858,105.482759,2.091954,41.355651,15.572845,...,3081.666667,2.555091e+06,316.000000,238.666667,62.0,62.000000,27.555873,57.898450,58.853776,51.452520
8,9,33572.200333,49.188857,62.358333,1.116667,21.509276,88.366667,0.266667,30.440556,9.228805,...,399.500000,1.007166e+06,320.000000,270.000000,66.0,44.000000,16.256250,46.483333,46.848485,9.079545
9,10,40953.386613,66.376464,82.145161,1.956613,26.458035,93.758065,1.629032,32.694086,12.745818,...,908.500000,1.269555e+06,341.000000,271.500000,64.5,43.000000,26.741256,39.821863,47.286797,20.715909


In [163]:
calmonthly.to_csv(out + pref + '_calmonthly.csv')

In [164]:
calmonthly_hours=hourly.groupby(['month','hour']).mean()
calmonthly_hours=calmonthly_hours.drop(['year','day','day_of_week'],1)
calmonthly_hours

price     volume       value
month hour                                  
1     0      9.887467  12.093333  394.868800
      1     11.724533  13.400000  453.762933
      2     12.176000  16.626667  562.506667
      3     12.974800  19.546667  660.124400
      4     13.805054  23.053763  762.579355
...               ...        ...         ...
12    19    19.870968   6.564516  536.048387
      20    16.822581   9.048387  595.500000
      21    20.080645  14.451613  894.032258
      22     6.806452   4.532258  182.338710
      23    10.066129  12.161290  449.335484

[288 rows x 3 columns]

In [165]:
calmonthly_hours.to_csv(out + pref + '_calmonthly_hours.csv')

In [166]:
yearly_calmonthly_hours=hourly.groupby(['year','month','hour']).mean()
yearly_calmonthly_hours=yearly_calmonthly_hours.drop(['day','day_of_week'],1)
yearly_calmonthly_hours

price     volume         value
year month hour                                    
2020 1     0      18.32000  26.680000    841.720000
           1      20.12000  26.640000    881.840000
           2      19.92000  30.960000   1016.120000
           3      23.72000  38.720000   1270.920000
           4      22.83871  45.451613   1441.516129
...                    ...        ...           ...
2022 8     19    154.66800  77.640000  20974.028400
           20    117.41120  75.440000  16784.211600
           21     52.49640  82.320000   6578.574000
           22     30.43520  74.560000   3182.585600
           23     20.54760  57.320000   1886.213200

[768 rows x 3 columns]

In [167]:
yearly_calmonthly_hours.to_csv(out + pref + '_yearly_calmonthly_hours.csv')

In [168]:
from statsmodels.tsa.stattools import pacf

In [169]:
pacfs=pd.DataFrame(data={'standard': pacf(monthly.reset_index()['standard_price_mean'], nlags=14)})
pacfs['evening']= pacf(monthly.reset_index()['evening_price_mean'], nlags=14)
pacfs['morning']= pacf(monthly.reset_index()['morning_price_mean'], nlags=14)
pacfs['off']= pacf(monthly.reset_index()['off_price_mean'], nlags=14)
pacfs

,standard,evening,morning,off
0,1.000000,1.000000,1.000000,1.000000
1,-0.013508,0.516127,0.137881,0.447301
2,0.192715,-0.146951,0.040538,0.146293
3,-0.080254,-0.137978,0.003362,-0.045187
4,-0.171353,-0.224530,0.029149,0.048562
5,-0.019103,0.044243,0.001037,-0.031555
6,-0.205611,-0.058838,-0.127555,-0.194716
7,0.145764,-0.016357,-0.115184,-0.282470
8,-0.348597,-0.137489,-0.330384,0.029254
9,0.078386,0.045655,0.219460,0.019404


In [170]:
pacfs.to_csv(out + pref + '_pacfs.csv')